In [1]:
from gurobi import *

In [2]:
rack_name, slots = multidict({
    'alpha' : 48
})

In [3]:
chips, quantity, powerdraw, slots_required = multidict({
    '1080' : [10, 45, 2],
    '2080' : [10, 55, 2]
})

In [4]:
m = Model("Tactus")

Academic license - for non-commercial use only


In [5]:
# always need the load on each phase Lead
L = m.addVars(3, name='phases')

In [22]:
# add x_ij where i is the chip and j is the possible orientations
# assume only 2 slots necessary
# orientation
# 1 = 1,2
# 2 = 1,3
# 3 = 2,3
# again the set is not an SOS
indicies = [(chip,orientation) for chip in chips for orientation in range(3)] # gen pairs 
x = m.addVars(indicies,vtype=GRB.INTEGER)

In [23]:
m.addConstr(quicksum(powerdraw[chip]*(x[chip,0]+x[chip,1]) for chip in chips),GRB.EQUAL,L[0]);
m.addConstr(quicksum(powerdraw[chip]*(x[chip,0]+x[chip,2]) for chip in chips),GRB.EQUAL,L[1]);
m.addConstr(quicksum(powerdraw[chip]*(x[chip,1]+x[chip,2]) for chip in chips),GRB.EQUAL,L[2]);

In [24]:
m.addConstrs(quicksum(x[chip,orient] for orient in range(3)) == quantity[chip] for chip in chips);

In [25]:
# simplifies later constraints
m.addConstr(L[0],GRB.LESS_EQUAL,L[1])
m.addConstr(L[1],GRB.LESS_EQUAL,L[2])

<gurobi.Constr *Awaiting Model Update*>

In [26]:
m.setObjective(L[2]-L[0])

In [27]:
m.optimize()

Optimize a model with 20 rows, 15 columns and 80 nonzeros
Variable types: 9 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 15 columns, 46 nonzeros
Variable types: 6 continuous, 9 integer (0 binary)
Found heuristic solution: objective 1000.0000000

Root relaxation: objective 0.000000e+00, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    7 1000.00000    0.00000   100%     -    0s
*    0     0               0       5.0000000    5.00000  0.00%     -    0s

Cutting planes:
  Gomory: 2
  MIR: 1
  Mod-K: 1

Explored 1 nodes (10 simplex iterations) in 0.04 seconds
Thread count was 8 (of 8 available processors)

So

In [28]:
x

{('1080', 0): <gurobi.Var C9 (value -0.0)>,
 ('1080', 1): <gurobi.Var C10 (value 5.0)>,
 ('1080', 2): <gurobi.Var C11 (value 5.0)>,
 ('2080', 0): <gurobi.Var C12 (value 6.0)>,
 ('2080', 1): <gurobi.Var C13 (value 2.0)>,
 ('2080', 2): <gurobi.Var C14 (value 2.0)>}

In [29]:
L

{0: <gurobi.Var phases[0] (value 665.0)>,
 1: <gurobi.Var phases[1] (value 665.0)>,
 2: <gurobi.Var phases[2] (value 670.0)>}